<span style="color:#1976d2; font-size:2.2em; font-weight:bold;">🔥 Projet London Fire Response</span>

<span style="color:#42a5f5; font-size:2em; font-weight:bold;">Exploration des données</span>

Ce notebook a pour but de charger, assembler et analyser rapidement les données brutes provenant de deux sources :

-les incidents signalés 

-les mobilisations des pompiers

<span style="color:#e91e63; font-size:1em; font-weight:bold;">1. Import des bibliothèques Python</span>

In [10]:
import os                # Pour manipuler les chemins de fichiers
import pandas as pd      # Pour charger et analyser les données
import glob              # Pour lister automatiquement tous les fichiers d’un dossier
import matplotlib.pyplot as plt  # Visualisation
import seaborn as sns             # Visualisation statistique

In [11]:
# Pour afficher toutes les colonnes et tout explorer !
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

  <span style="color:#e91e63; font-size:1em; font-weight:bold;">2. Définition les chemins propres</span>

In [13]:
# Revenir au dossier parent
base_path = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Dossiers contenant les fichiers incidents et mobilisations
incidents_path = os.path.join(base_path, "data", "raw", "incidents")
mobilisations_path = os.path.join(base_path, "data", "raw", "mobilisations")

print("-Chemin incidents :", incidents_path)
print("-Chemin mobilisations :", mobilisations_path)

-Chemin incidents : c:\Users\9609241C\london-fire-response\data\raw\incidents
-Chemin mobilisations : c:\Users\9609241C\london-fire-response\data\raw\mobilisations


<span style="color:#e91e63; font-size:1em; font-weight:bold;">3. Charger tous les fichiers incidents (.csv)  </span>

In [14]:
incident_files = glob.glob(os.path.join(incidents_path, "*.csv"))

# On charge tous les fichiers et on les regroupe
incidents = pd.concat([pd.read_csv(f) for f in incident_files], ignore_index=True)

# Aperçu rapide
print("Données incidents chargées !")
print(incidents.head())
print("-Nombre de lignes :", len(incidents))
print("-Nombre de colonnes :", len(incidents.columns))
print("-Colonnes :", incidents.columns.tolist())
print("-Types de données :", incidents.dtypes)

C:\Users\9609241C\AppData\Local\Temp\ipykernel_3596\233953251.py:4: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  incidents = pd.concat([pd.read_csv(f) for f in incident_files], ignore_index=True)


Données incidents chargées !
  IncidentNumber DateOfCall  CalYear TimeOfCall  HourOfCall    IncidentGroup  \
0    235138081.0  01-Jan-09     2009   00:00:37           0  Special Service   
1         1091.0  01-Jan-09     2009   00:00:46           0  Special Service   
2         2091.0  01-Jan-09     2009   00:03:00           0             Fire   
3         3091.0  01-Jan-09     2009   00:04:27           0             Fire   
4         5091.0  01-Jan-09     2009   00:05:39           0             Fire   

  StopCodeDescription     SpecialServiceType PropertyCategory  \
0     Special Service                    RTC     Road Vehicle   
1     Special Service  Assist other agencies          Outdoor   
2      Secondary Fire                    NaN          Outdoor   
3      Secondary Fire                    NaN          Outdoor   
4      Secondary Fire                    NaN          Outdoor   

                                  PropertyType  \
0                                         Car    

> ## 📊 Observations de l'importation et exploration initiale des données `incidents`
* Les données ont bien été chargées (environ 1,8 million de lignes, 39 colonnes).
* Avertissement sur `IncidentNumber` : mélange de types (probablement des entiers et des chaînes).
* Beaucoup de colonnes sont de type texte (`object`), avec aussi des numériques (`int64`, `float64`), notamment pour les coûts, le nombre de pompes, les temps d’intervention, etc.
* Il y a plusieurs colonnes temporelles redondantes (`DateOfCall`, `TimeOfCall`, `HourOfCall`, `CalYear`) → à combiner.
* Les colonnes liées à la localisation sont riches : code postal, latitude/longitude, boroughs, wards…
* On observe des valeurs manquantes dans certaines colonnes (`UPRN`, `USRN`, `SecondPump...`).
* Quelques colonnes semblent redondantes ou doublées (`IncGeo_WardName` vs `IncGeo_WardNameNew`, `FRS` vs `ProperCase`).
* Il faudra convertir les dates en datetime, harmoniser les types, et potentiellement nettoyer les colonnes peu remplies ou inutiles.

<span style="color:#e91e63; font-size:1em; font-weight:bold;">4. Charger tous les fichiers mobilisations (.csv ou .xlsx) </span>

In [15]:
# Fichiers .csv et .xlsx
mobilisation_csv = glob.glob(os.path.join(mobilisations_path, "*.csv"))
mobilisation_xlsx = glob.glob(os.path.join(mobilisations_path, "*.xlsx"))

# Charger tout et regrouper
mobilisations = pd.concat(
    [pd.read_csv(f) for f in mobilisation_csv] + 
    [pd.read_excel(f) for f in mobilisation_xlsx],
    ignore_index=True
)

print(" Données mobilisations chargées !")
print(mobilisations.head())

C:\Users\9609241C\AppData\Local\Temp\ipykernel_3596\230581493.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mobilisations = pd.concat(


 Données mobilisations chargées !
    IncidentNumber  CalYear           BoroughName      WardName  HourOfCall  \
0  000004-01012021     2021              HARINGEY  Muswell Hill           0   
1  000005-01012021     2021             REDBRIDGE      MONKHAMS           0   
2  000006-01012021     2021  BARKING AND DAGENHAM       Village           0   
3  000007-01012021     2021            WANDSWORTH     West Hill           0   
4  000007-01012021     2021            WANDSWORTH     West Hill           0   

   ResourceMobilisationId Resource_Code PerformanceReporting  \
0                 5769249          A321                    1   
1                 5769250          F351                    1   
2                 5769251          F412                    1   
3                 5769252          H331                    1   
4                 5769253          G351                    2   

  DateAndTimeMobilised DateAndTimeMobile DateAndTimeArrived  \
0     01/01/2021 00:06  01/01/2021 00:07   

In [16]:
#Suite exploration mobilisations
print("-Nombre de lignes :", len(mobilisations))
print("-Nombre de colonnes :", len(mobilisations.columns))
print("-Colonnes :", mobilisations.columns.tolist())
print("-Types de données :", mobilisations.dtypes)

-Nombre de lignes : 2561435
-Nombre de colonnes : 24
-Colonnes : ['IncidentNumber', 'CalYear', 'BoroughName', 'WardName', 'HourOfCall', 'ResourceMobilisationId', 'Resource_Code', 'PerformanceReporting', 'DateAndTimeMobilised', 'DateAndTimeMobile', 'DateAndTimeArrived', 'TurnoutTimeSeconds', 'TravelTimeSeconds', 'AttendanceTimeSeconds', 'DateAndTimeLeft', 'DateAndTimeReturned', 'DeployedFromStation_Code', 'DeployedFromStation_Name', 'DeployedFromLocation', 'PumpOrder', 'PlusCode_Code', 'PlusCode_Description', 'DelayCodeId', 'DelayCode_Description']
-Types de données : IncidentNumber                      object
CalYear                              int64
BoroughName                         object
WardName                            object
HourOfCall                           int64
ResourceMobilisationId               int64
Resource_Code                       object
PerformanceReporting                object
DateAndTimeMobilised                object
DateAndTimeMobile                   obj

> ## 📊 Observations de l'importation et exploration initiale des données `mobilisations`
- d'apres le warning du chargement avec pd : présence de colonnes vides ou très mal renseignées dans certains fichiers du dataset.
-`DelayCode_Description` contient beaucoup de valeurs manquantes
- Faudra faire attention à la conversion des `IncidentNumber` notamment pr le join entre les 2 datasets !
- Les 4 colonnes temporelles 'DateAndTimeMobilised','DateAndTimeMobile' ,'DateAndTimeArrived' ,'DateAndTimeLeft': ont un format cohérent nécessitent un to_datetime().

IncidentNumber : etant  l'identifiant de l'incident
PumpOrder : ordre d’arrivée de l’engin 
ApplianceIdentifier / ApplianceCallSign : identifiants du véhicule
==> Plusieurs mobilisations sont donc liées à un même incident, ce qui créera des doublons d’incidents mais lignes uniques de mobilisation.

Pour ce qui des données de Localisation :
BoroughCode / BoroughName, DeployedFromStation_Code / DeployedFromStation_Name, StationGround
==> Les données  semblentmajoritairement complètes sur la géographie, avec des noms lisibles. Peu de valeurs nulles.

<span style="color:#e91e63; font-size:1em; font-weight:bold;">5. Jointure entre les data incidents et mobilisations </span>

In [19]:
# Harmoniser IncidentNumber dans les deux DataFrames
incidents['IncidentNumber'] = incidents['IncidentNumber'].astype(str).str.strip()
mobilisations['IncidentNumber'] = mobilisations['IncidentNumber'].astype(str).str.strip()

In [20]:
# Jointure sur l'ID IncidentNumber
df_jointure = mobilisations.merge(incidents, on="IncidentNumber", how="inner")

In [21]:
#Aperçu de la jointure:
df_jointure.head()
df_jointure.info()
df_jointure.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1538089 entries, 0 to 1538088
Data columns (total 62 columns):
 #   Column                                  Non-Null Count    Dtype         
---  ------                                  --------------    -----         
 0   IncidentNumber                          1538089 non-null  object        
 1   CalYear_x                               1538089 non-null  int64         
 2   BoroughName                             773072 non-null   object        
 3   WardName                                772197 non-null   object        
 4   HourOfCall_x                            1538089 non-null  int64         
 5   ResourceMobilisationId                  1538089 non-null  int64         
 6   Resource_Code                           1538089 non-null  object        
 7   PerformanceReporting                    1538089 non-null  object        
 8   DateAndTimeMobilised                    1538089 non-null  object        
 9   DateAndTimeMobile       

Index(['IncidentNumber', 'CalYear_x', 'BoroughName', 'WardName',
       'HourOfCall_x', 'ResourceMobilisationId', 'Resource_Code',
       'PerformanceReporting', 'DateAndTimeMobilised', 'DateAndTimeMobile',
       'DateAndTimeArrived', 'TurnoutTimeSeconds', 'TravelTimeSeconds',
       'AttendanceTimeSeconds', 'DateAndTimeLeft', 'DateAndTimeReturned',
       'DeployedFromStation_Code', 'DeployedFromStation_Name',
       'DeployedFromLocation', 'PumpOrder', 'PlusCode_Code',
       'PlusCode_Description', 'DelayCodeId', 'DelayCode_Description',
       'DateOfCall', 'CalYear_y', 'TimeOfCall', 'HourOfCall_y',
       'IncidentGroup', 'StopCodeDescription', 'SpecialServiceType',
       'PropertyCategory', 'PropertyType', 'AddressQualifier', 'Postcode_full',
       'Postcode_district', 'UPRN', 'USRN', 'IncGeo_BoroughCode',
       'IncGeo_BoroughName', 'ProperCase', 'IncGeo_WardCode',
       'IncGeo_WardName', 'IncGeo_WardNameNew', 'Easting_m', 'Northing_m',
       'Easting_rounded', 'Northing_

In [22]:
# Affichage de toutes les colonnes
print(df_jointure.columns.tolist())

['IncidentNumber', 'CalYear_x', 'BoroughName', 'WardName', 'HourOfCall_x', 'ResourceMobilisationId', 'Resource_Code', 'PerformanceReporting', 'DateAndTimeMobilised', 'DateAndTimeMobile', 'DateAndTimeArrived', 'TurnoutTimeSeconds', 'TravelTimeSeconds', 'AttendanceTimeSeconds', 'DateAndTimeLeft', 'DateAndTimeReturned', 'DeployedFromStation_Code', 'DeployedFromStation_Name', 'DeployedFromLocation', 'PumpOrder', 'PlusCode_Code', 'PlusCode_Description', 'DelayCodeId', 'DelayCode_Description', 'DateOfCall', 'CalYear_y', 'TimeOfCall', 'HourOfCall_y', 'IncidentGroup', 'StopCodeDescription', 'SpecialServiceType', 'PropertyCategory', 'PropertyType', 'AddressQualifier', 'Postcode_full', 'Postcode_district', 'UPRN', 'USRN', 'IncGeo_BoroughCode', 'IncGeo_BoroughName', 'ProperCase', 'IncGeo_WardCode', 'IncGeo_WardName', 'IncGeo_WardNameNew', 'Easting_m', 'Northing_m', 'Easting_rounded', 'Northing_rounded', 'Latitude', 'Longitude', 'FRS', 'IncidentStationGround', 'FirstPumpArriving_AttendanceTime', '

-Colonnes avec valeurs manquantes
Certaines colonnes ont un nombre important de valeurs manquantes, par exemple :
BoroughName, WardName, DateAndTimeReturned seulement 4 valeurs non-nulles : à vérifier, peut-être colonne à ignorer.
DelayCodeId et DelayCode_Description,Postcode_full, Latitude, Longitude ont aussi beaucoup de manquants 

-Colonnes à doublons ou similaire
Tu as des colonnes CalYear_x et CalYear_y, HourOfCall_x et HourOfCall_y : 
-> faut vérifier si leur contenu est idem pour n'en garder 1 !

-Types de données
Date/heure stockées en object pour plusieurs colonnes (DateAndTimeMobilised, .. ==> à formater!

-Suggestions d’actions possibles
Nettoyage